In [ ]:
import numpy as np
import mdtraj as md
import nglview as nv
from main import make_bins, get_neighbor_bins
from utils import load_data

In [2]:
# load receptor
receptor_traj = md.load('data/1a52_cleaned.pdb')
view = nv.show_mdtraj(receptor_traj)
view.add_representation('cartoon')  # or cartoon

# load ligand
ligand  = md.load("data/estradiol.pdb")
view.add_component(ligand)

In [3]:
nv.write_html('receptor_ligand_complex.html', [view])

frame_range is not provided. Do not serialize coordinates. Default to use current frame.


In [4]:
def cube_edges(center, size):
    """
    Get all the edges for each bin (cube)
    """
    cx, cy, cz = center
    r = size / 2

    # cube corners
    p = [
        (cx-r, cy-r, cz-r),  # 0
        (cx+r, cy-r, cz-r),  # 1
        (cx+r, cy+r, cz-r),  # 2
        (cx-r, cy+r, cz-r),  # 3
        (cx-r, cy-r, cz+r),  # 4
        (cx+r, cy-r, cz+r),  # 5
        (cx+r, cy+r, cz+r),  # 6
        (cx-r, cy+r, cz+r),  # 7
    ]

    # edges: pairs of corner indices
    e = [
        (0,1),(1,2),(2,3),(3,0),
        (4,5),(5,6),(6,7),(7,4),
        (0,4),(1,5),(2,6),(3,7),
    ]

    # return list of edges as point pairs
    return [(p[i], p[j]) for i,j in e]

In [5]:
ligand_coords, receptor_coords = load_data()

In [6]:
def get_used_bins(ligand, receptor_bins, bin_size=4.0, cutoff=5.0):
    """
    Return a set of bin keys that were actually used in scoring.
    """
    used_bins = set()
    radius_bins = int(np.ceil(cutoff / bin_size))

    for pos in ligand:
        key = tuple((pos // bin_size).astype(int))
        for neighbor_key in get_neighbor_bins(key, radius_bins):
            if neighbor_key in receptor_bins:
                used_bins.add(neighbor_key)

    return used_bins

In [7]:
def collect_unique_edges(bins, bin_size):
    """
    Deduplicates bin edges to make it easier to add them
    and makes viewing less laggy.
    """
    unique_edges = set()
    edges_out = []

    for key in bins:
        center = (np.array(key) + 0.5) * bin_size
        edges = cube_edges(center, bin_size)

        for p1, p2 in edges:
            # canonical key: sort the endpoints
            key_edge = tuple(sorted((p1, p2)))
            if key_edge not in unique_edges:
                unique_edges.add(key_edge)
                edges_out.append((p1, p2))

    return edges_out

In [8]:
import tqdm
def draw_edge_list(view, edges, color=[1,1,0], radius=0.1):
    for p1, p2 in tqdm.tqdm(edges):
        view.shape.add_cylinder(list(p1), list(p2), color, radius)

In [9]:
bin_size = 4.0
bins = make_bins(receptor_coords, bin_size=bin_size)
bins = get_used_bins(ligand_coords, bins)
edges = collect_unique_edges(bins, bin_size)
print(len(edges))
draw_edge_list(view, edges)

844


100%|███████████████████████████████████████████████████████████████████████| 844/844 [01:57<00:00,  7.19it/s]


In [10]:
view

NGLWidget()